<a href="https://colab.research.google.com/github/neurontist/LangChain-for-LLM-Application-Development/blob/main/L7%20How%20To%20Use%20Structured%20Outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How To Use Structured Outputs

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google import genai
from pydantic import BaseModel
from typing import Optional

In [ ]:
import os
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

### Define structure with Pydantic

In [ ]:
model = "gemini-2.0-flash"

In [ ]:
class User(BaseModel):
  name:str
  age:int
  email:Optional[str] = None

In [ ]:
response = client.models.generate_content(
    model=model,
    contents = [
        {"role": "model", "parts": [{"text":"You are a helpful assistant."}]},
        {"role": "user", "parts": [{"text":"Make up a user."}]},
    ],
    config={
        "response_mime_type":"application/json",
        "response_schema":User,
    },
)

print(response.text)
user= response.parsed
print(user.name)

{
  "name": "John Doe",
  "age": 30,
  "email": "john.doe@example.com"
}
John Doe


### The social media mention structure

In [ ]:
from pydantic import BaseModel
from enum import Enum
from typing import List,Optional,Literal
from google import genai

### Creating JSON Response Schema using **Pydantic**

In [ ]:
class Mention(BaseModel):
  # The model choose the product the mention is about
  product: Literal['app','website','not_applicable']
  # The sentiment of the mention
  sentiment: Literal['negative','positive','neutral']
  # If the model needs to respond to the user
  needs_response: bool
  response: Optional[str]
  # If a support ticket needs to be opened and the model can write the description
  support_ticket_decsp: Optional[str]

### Using some examples to work with

In [ ]:
# Example mentions
mentions = [
    # About the app
    "@techcorp your app is amazing! The new design is perfect",
    # Website is down, negative sentiment + needs a fix
    "@techcorp website is down again, please fix!",
    # Nothing to respond to
    "hey @techcorp you're so evil"
]

### Let's analyze the mentions as we want

In [ ]:
def analyze_mention(mention:str,personality:str = "friendly") -> Mention:
  completion = client.models.generate_content(
      model=model,
      contents = [
          {"role":"model",
           "parts":[{"text":
                     f"""
           Extract structured information from
                social media mentions about our products.

                Provide
                - The product mentioned (website, app, not applicable)
                - The mention sentiment (positive, negative, neutral)
                - Whether to respond (true/false). Don't respond to
                  inflammatory messages or bait.
                - Respond to positive mentions with a friendly message.
                - A customized response to send to the user if we need
                  to respond.
                - An optional support ticket description to create.

                Your personality is {personality}.

           """}]},
          {"role":"user",
           "parts":[{"text":mention}]}
      ],
      config={
        "response_mime_type":"application/json",
        "response_schema":Mention,
    },
  )
  return completion

In [ ]:
print("User post:", mentions[0])
processed_mention = analyze_mention(mentions[0])
processed_mention.parsed.needs_response

User post: @techcorp your app is amazing! The new design is perfect


True

## Programming with our mention

In [ ]:
def print_mention(processed_mention,mention):
  if processed_mention.parsed.needs_response:
    print(f"Responding to {processed_mention.parsed.sentiment} {processed_mention.parsed.product} feedback")
    print(f"  User: {mention}")
    print(f"  Response: {processed_mention.parsed.response}")
  else:
      print(f"Not responding to {processed_mention.parsed.sentiment} {processed_mention.parsed.product} post")
      print(f"  User: {mention}")
  if processed_mention.parsed.support_ticket_decsp:
    print(f"  Support ticket description: {processed_mention.parsed.support_ticket_decsp}")

### Let's apply analyze to each mention and see how LLM responds to each

In [ ]:
rows = []

for mention in mentions:
  processed_mention = analyze_mention(mention)

  print_mention(processed_mention,mention)
  # converting processed_mention to dictionary to add mention key and the value
  processed_mention_dict = processed_mention.model_dump()
  processed_mention_dict['mention'] = mention

  rows.append(processed_mention_dict)
  print("")

Responding to positive app feedback
  User: @techcorp your app is amazing! The new design is perfect
  Response: Thanks so much for the kind words! We're so glad you're enjoying the new design.

Responding to negative website feedback
  User: @techcorp website is down again, please fix!
  Response: We're so sorry you're having trouble accessing the website! Our team is on it and working hard to get it back up and running. We appreciate your patience!
  Support ticket description: User reports website is down.

Not responding to negative not_applicable post
  User: hey @techcorp you're so evil



### Representing in tabular format

In [ ]:
import pandas as pd

# Convert the 'rows' list into a Pandas DataFrame
df = pd.DataFrame()
for row in rows:
  df = pd.concat([df,pd.DataFrame([row.get('parsed')])])

df

,product,sentiment,needs_response,response,support_ticket_decsp
0,app,positive,True,Thanks so much for the kind words! We're so gl...,None
0,website,negative,True,We're so sorry you're having trouble accessing...,User reports website is down.
0,not_applicable,negative,False,None,None


In [ ]:
mention_json_string = processed_mention.model_dump()
print(mention_json_string.get('parsed'))

{'product': 'app', 'sentiment': 'positive', 'needs_response': True, 'response': "We're so glad you like the new design!", 'support_ticket_decsp': None}


### Try it yourself!

In [ ]:
class UserPost(BaseModel):
  response:str

def makePost(output_class):
  completion = client.models.generate_content(
      model=model,
      contents = [
          {"role": "model", "parts": [{"text":f"""
                You are a customer of Tech Corp (@techcorp), a company
                that provides an app and a website. Create a small
                microblog-style post to them that sends some kind of
                feedback, positive or negative.
            """}]},
            {"role": "user", "parts": [{"text":"Please write a post."}]},
      ],
      config = {
          "response_mime_type":"application/json",
          "response_schema":output_class
      }
  )
  return completion

In [ ]:
user_post = makePost(UserPost)
user_post.parsed

UserPost(response='Love the app, especially the new dark mode! But the website still feels a bit clunky on mobile. Any plans to update it?')

In [ ]:
class UserPostWithExtras(BaseModel):
  user_mood:Literal['evil','bad','awful','good']
  product:Literal['app','website','not applicable']
  sentiment:Literal['positive','negative','neutral']
  internal_monologue:List[str]
  response:str

new_post = makePost(UserPostWithExtras)
new_post.parsed

UserPostWithExtras(user_mood='bad', product='app', sentiment='negative', internal_monologue=['Ugh, this app update is terrible.', 'Why did they change the layout?', "It's so confusing now.", 'I wish they would just revert to the old version.'], response='@techcorp New app update is awful! The layout is confusing and hard to navigate. Please go back to the old version!')